In [291]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
device = T.device("mps")

In [9]:
appa_hours = pd.read_csv("exports/appa_hr_crop_withCreated.csv")

In [34]:
appa_hours = appa_hours[['ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC',
       'ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated', 'NO2', 'SO2', 'O3', 'CO']]

In [749]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,median_absolute_error, mean_absolute_error

In [595]:
appa_hours.columns

Index(['ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC',
       'ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated'],
      dtype='object')

In [27]:
cols = ['ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC',
       'ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated']
outs = ['NO2', 'O3', 'CO']

In [756]:
vsplit = round(len(appa_hours) * 0.2)

X_train = appa_hours.iloc[vsplit:][cols]
X_test = appa_hours.iloc[:vsplit][cols]

Y_train = appa_hours.iloc[vsplit:][outs]
Y_test = appa_hours.iloc[:vsplit][outs]

In [752]:
model = RandomForestRegressor()
model.fit(X_train, Y_train)

/var/folders/gl/2h8nn4ps031g1jysgll17wz00000gn/T/ipykernel_13826/1006863496.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, Y_train)


RandomForestRegressor()

In [754]:
Y_test_pred_r = model.predict(X_test)

print("Random Forest")
print(f"Median Error: {median_absolute_error(Y_test, Y_test_pred_r)} \n Mean error: {mean_absolute_error(Y_test, Y_test_pred_r)}")

Random Forest
Median Error: 10.86 
 Mean error: 12.902733576642337


# Transformers

In [312]:
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [11]:
appa_hours.columns

Index(['Time', 'ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC', 'PM10', 'NO2',
       'SO2', 'O3', 'CO', 'ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated'],
      dtype='object')

In [13]:
len(['Time', 'ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC','ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated'])

29

In [569]:
X.shape

(13678, 24, 28)

In [643]:
class BasicNet(nn.Module):
    def __init__(self, in_channel):
        super(BasicNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=20, kernel_size=(4,4))

        self.conv2 = nn.Conv2d(in_channels=20, out_channels=3, kernel_size=(4, 4))

        self.fc1 = nn.Linear(1188, 200)
        self.fc2 = nn.Linear(200, 150)
        self.fc3 = nn.Linear(150, 50)
        self.fc4 = nn.Linear(50, 3)

        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
    def forward(self, x):

        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.fc1(torch.flatten(x))
        x = F.relu(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = self.dropout1(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x

basic_model = BasicNet(1)
print(basic_model)

BasicNet(
  (conv1): Conv2d(1, 20, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(20, 3, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=1188, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=150, bias=True)
  (fc3): Linear(in_features=150, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=3, bias=True)
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
)


In [667]:
basic_model(torch.tensor(X[0]).float())

tensor([-0.0022,  0.1585, -0.0574], grad_fn=<AddBackward0>)

In [582]:
X = appa_hours[['ZnOR_1', 'ZnOR_2', 'LaFeO3_1', 'LaFeO3_2', 'WO3_1', 'WO3_2',
       'ZnOR_heatR_1', 'ZnOR_heatR_2', 'LaFeO3_heatR_1', 'LaFeO3_heatR_2',
       'WO3_heatR_1', 'WO3_heatR_2', 'ZnOR_heatV_1', 'ZnOR_heatV_2',
       'LaFeO3_heatV_1', 'LaFeO3_heatV_2', 'WO3_heatV_1', 'WO3_heatV_2',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC',
       'ZnOR_1_TimeSinceCreated', 'ZnOR_2_TimeSinceCreated',
       'LaFeO3_1_TimeSinceCreated', 'LaFeO3_2_TimeSinceCreated',
       'WO3_1_TimeSinceCreated', 'WO3_2_TimeSinceCreated']].to_numpy()
Y = appa_hours[['NO2', 'O3', 'CO']].to_numpy()

In [583]:
scalerX = preprocessing.StandardScaler().fit(X)
X = scalerX.transform(X)
scalerY = preprocessing.StandardScaler().fit(Y)
Y = scalerY.transform(Y)

In [584]:
X = np.lib.stride_tricks.sliding_window_view(X, (24,28))
Y = Y[X.shape[1]-1:]
(X.shape,Y.shape)

((13678, 1, 24, 28), (13701, 3))

In [585]:
data = []
for i in range(len(X)):
    data.append([X[i], Y[i]])

In [586]:
vsplit = round(len(data) * 0.2)

train_dl = DataLoader(data[vsplit:], batch_size=32, shuffle=True)
test_dl = DataLoader(data[:vsplit], batch_size=32, shuffle=True)

In [670]:
def train_model(train_dl, model):
    global bm
    best_loss = float("inf")
    # define the optimization
    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(basic_model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            for i, inp in enumerate(inputs):
                yhat = model(inp.float())
                # calculate loss
                loss = criterion(yhat, targets[i].float())
                # credit assignment
                loss.backward()
            # update model weights
            optimizer.step()
        if best_loss > loss:
            bm = copy.deepcopy(model)
            best_loss = loss
        print(f"epoch {epoch} done, loss {loss}")

In [602]:
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        for i, inp in enumerate(inputs):
            yhat = model(inp.float())
            # retrieve numpy array
            yhat = yhat.detach().numpy()
            actual = targets[i].numpy()
            # store
            predictions.append(yhat)
            actuals.append(actual)
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    # calculate accuracy
    acc = (actuals, predictions)
    return acc

In [187]:
import copy

In [669]:
train_model(train_dl, basic_model)

epoch 0 done, loss 0.21166719496250153


KeyboardInterrupt: 

In [642]:
basic_model(torch.rand((1,24,28)))

tensor([-0.0576, -0.0222, -0.0243], grad_fn=<AddBackward0>)

In [630]:
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_error
evals = evaluate_model(test_dl, basic_model)

median_absolute_error(scalerY.inverse_transform(evals[1]),  scalerY.inverse_transform(evals[0]))

14.764513163195716

In [631]:
evals[1]

array([[-0.05760721, -0.02216033, -0.02431159],
       [-0.05760721, -0.02216033, -0.02431159],
       [-0.05760721, -0.02216033, -0.02431159],
       ...,
       [-0.05760721, -0.02216033, -0.02431159],
       [-0.05760721, -0.02216033, -0.02431159],
       [-0.05760721, -0.02216033, -0.02431159]], dtype=float32)

In [459]:
def extract_windows(array, clearing_time_index, max_time, sub_window_size):
    examples = []
    start = clearing_time_index + 1 - sub_window_size + 1
    
    for i in range(max_time+1):
        example = array[start+i:start+sub_window_size+i]
        examples.append(np.expand_dims(example, 0))
    
    return np.vstack(examples)